# Multiple Regression
In the last lab and in the homework, we saw how we can model the response of a voxel $v$ to a stimulus $i$ via the hemodynamic response function, and a weight $\beta^v_i$. To do this we assume that we know the shape of the hemodynamic response, and we assume a value for how well a voxel responds to a stimulus. But in reality, we do not know these. The hemodynamic response has been a subject for a lot of research and therefore it is common to use a canonical function like the one we explored last week. However, estimating how much a voxel responds to a specific stimulus is exactly what the purpose of an fMRI experiment is.

# Goals

- The regression problem and it's solution
- Linear regression with real data
- Multiple regression
- Contrast maps in the assignment!

In [ ]:
# Imports
import neurods as nds
import numpy as np
import scipy
import os
import matplotlib.pyplot as plt
# Configure defaults for plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'
%matplotlib widget

First let's talk about linear regression, and start to work with a simple problem. Let's say you walk into a grocery store, and you buy 3 oranges and 2 apples. You are told the price is 8$\$$. You go another time, you buy 2 oranges and 5 apples, you pay 9$\$$. How much do apples and oranges cost?

This is a system of two equations with two unknown that you can solve and obtain an exact solution.

However, imagine that the cashier doesn't tell you the exact price, but takes the correct price, and then depending on their mood, adds some "noise" to the price: you either pay a little less or a little more. This noise doesn't depend on how much your total is, but on other unrelated factors.

Can you still estimate accurately the prices if you go twice to the store?

What if you go 1000 times?

Let's simulate this with a random sample:

In [ ]:
# these are the hidden parameters, we are not supposed to know them:
apple_price = 0.9
orange_price = 1.2
pear_price = 1.5
noise_variance = 5

# sample X and Y:
n = 1000
X = np.round(np.random.uniform(low = 0, high = 10, size=[n,3])).astype(int)
real_beta = np.array([apple_price, orange_price, pear_price]).reshape([3,1])
Y = np.dot(X, real_beta) + np.random.normal(size =[n,1] )*noise_variance

We can express the price in one visit j as:

\begin{align}
y_j =  X_j \beta +\epsilon_j
\end{align}

Were the row $X_j = [x^a_j,x^o_j,x^p_j]$ corresponds to the counts $x^a_j,x^o_j,x^p_j$ of apples, oranges and pears bought on visit $j$, and $\beta = [\beta_a,\beta_o,\beta_p]$ corresponds to the prices  $\beta_a,\beta_o,\beta_p$ of apples, oranges and pears. 

We can write the entire visits as:

\begin{align}
Y =  {\bf X} \beta +\epsilon
\end{align}

where
- $Y$ is n x 1
- ${\bf X}$ is n x d, here d = 3
- ${\beta}$ is d x 1
- $\epsilon$ is n x 1.

Due to the noise, we cannot exactly recover $\beta$. However, we would like to find a solution $\hat\beta$ that minimizes the following error as much as possible:

\begin{align}
error = \sum_{j = 1}^N (y_j - X_j \beta)^2 = ||Y - {\bf X} \beta||_2^2
\end{align}

This is the sum of squared errors. To minimize this equation with respect to $\beta$, we first find the derivative with respect to $\beta$:

\begin{align}
\frac{\delta \ error}{\delta \beta} &=& \frac{\delta ||Y - {\bf X} \beta||_2^2}{\delta \beta}\\
 &=& -2{\bf X}^\top (Y - {\bf X} \beta)\\
\end{align}

The minimum is achieved when the derivative is zero:

\begin{align}
-2{\bf X}^\top (Y - {\bf X} \hat\beta) = 0\\
{\bf X}^\top Y = {\bf X}^\top{\bf X} \hat\beta\\
\hat\beta = ({\bf X}^\top{\bf X})^{-1}{\bf X}^\top Y\\
\end{align}

This is the Ordinary Least Squares Solution. Now write it as a function, then use this function to recover the prices of the fruits, using the following cell:

### BREAKOUT SESSION

In [ ]:
from numpy.linalg import inv
def OLS(X,Y):
### STUDENT ANSWER

In [ ]:
prices = OLS(X,Y)
print("apple price: real {0} estimated {1}".format(apple_price,prices[0,0]))
print("orange price: real {0} estimated {1}".format(orange_price,prices[1,0]))
print("pear price: real {0} estimated {1}".format(pear_price,prices[2,0]))

Try to change the number of datapoints and the magnitude of the noise, what do you notice?

## Intercept Term

Many times, we are interested in modeling $y_j$ as:

\begin{align}
y_j =  \beta_0 + \beta_1 x^1_j + \beta_2 x^2_j ... + \beta_3 x^d_j +\epsilon_j
\end{align}

this means there is a constant intercept term which is always contributed to the output $y_j$. In our store analogy, this could be for example an additional flat fare that is added by the cashier for each costumer. How can we integrate the intercept term in our framework?

There is a simple way, notice how we can rewrite the above equation as:

\begin{align}
y_j =  \beta_0 x^0_j+ \beta_1 x^1_j + \beta_2 x^2_j ... + \beta_3 x^d_j +\epsilon_j
\end{align}

where $x^0_j$ is always equal to 1. This can be done by creating a matrix $X'$ by adding an additional column to our matrix $X$ which is all ones. Let's try to estimate the intercept term in our fruit example:


In [ ]:
X2 = np.hstack([np.ones([n,1]),X])
prices = OLS(X2,Y)
print('intercept term is estimated to be {0}'.format(prices[0,0]))
print("apple price: real {0} estimated {1}".format(apple_price,prices[1,0]))
print("orange price: real {0} estimated {1}".format(orange_price,prices[2,0]))
print("pear price: real {0} estimated {1}".format(pear_price,prices[3,0]))

The intercept term is estimated to be 0, which makes sense because we didn't specify an intercept term! Let's sample data another time and estimate the intercept again:

In [ ]:
intercept_term = 2
Y2 = np.dot(X, real_beta) + np.random.normal(size =[n,1] )*noise_variance + intercept_term

prices = OLS(X2,Y2)
print('intercept term is estimated to be {0}'.format(prices[0,0]))
print("apple price: real {0} estimated {1}".format(apple_price,prices[1,0]))
print("orange price: real {0} estimated {1}".format(orange_price,prices[2,0]))
print("pear price: real {0} estimated {1}".format(pear_price,prices[3,0]))

When we work with fMRI data, we usually remove the mean of each voxel in the begining of our analysis. This means we don't need to include the intercept term in our design matrix, because it's effectively equal to zero.

## Modeling voxel responses



Remember, we are using regression because we want to model different voxel responses to a set of stimuli. We learned how to take a stimulus time course and how to convolve it with the hemodynamic response. We then assumed that each voxel's activity was a linear combination of all the convolved time courses of the stimuli. We want to recover the parameters of the linear combination. Let's load some data:

In [ ]:
basedir = '../../data/fMRI/categories/'
design = np.load(os.path.join(basedir,'experiment_design.npz'))
print('Experiment design variables: ', design.keys())
conditions = design['conditions'].tolist()
print('Conditions: ', conditions)
design_run1 = design['run1']
plt.figure()
for i, (cond, label) in enumerate(zip(design_run1.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)
plt.title('Condition labels')
_ = plt.legend(frameon=False, bbox_to_anchor=(1.4, 1))
plt.show()

We are going to use the zscore function while loading the data to normalize every block:

In [ ]:
import nibabel
from scipy.stats import zscore

def load_data(fname, mask=None, standardize=False):
    """Load fMRI data from nifti file, optionally with masking and standardization"""
    if isinstance(fname, (list, tuple)):
        return np.vstack([load_data(f, mask=mask, standardize=standardize) for f in fname])
    nii = nibabel.load(fname)
    data = nii.get_fdata().T.astype(np.float32)
    if mask is not None:
        data = data[:, mask]
    if standardize:
        data = scipy.stats.zscore(data, axis=0)
    return data

In [ ]:
import cortex

sub, xfm = 's01', 'catloc'

mask = cortex.db.get_mask(sub, xfm, type='cortical')
fname = os.path.join(basedir, 's01_categories_{n}.nii.gz') 
# fmri responses in all runs:
Y = np.vstack([zscore(load_data(fname.format(n=n), mask=mask, standardize=True)) for n in ['01', '02', '03']])
# stimuli:
X = np.vstack([design[run] for run in ['run1','run2', 'run3']])

plt.figure(figsize=(10,4))
plt.imshow(Y.T)
plt.title('Voxel responses')
plt.show()

plt.figure(figsize=(10,4))
for i, (cond, label) in enumerate(zip(X.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)
plt.title('Condition labels')
_ = plt.legend()
plt.show()

We need to first build a design matrix that accounts for the hemodynamic response:

In [ ]:
from neurods.fmri import hrf as generate_hrf
t_hrf, hrf_1 = generate_hrf(tr=2)
n, d = X.shape

conv_X = np.zeros_like(X, dtype=float)
for i in range(d):
    conv_X[:,i] = np.convolve(X[:,i], hrf_1)[:n]

plt.figure()
for i, (cond, label) in enumerate(zip(conv_X.T, conditions)):
    plt.plot(cond+i+0.2*i, label=label, lw=2)
    
plt.title('Condition labels')   
plt.legend()
plt.show()

## Linear Regression for real fMRI data


We want to find the response of all the voxels in the brain to these 5 different conditions. Instead of a one dimensional output $Y$, we have a high dimensional output ${\bf Y}$.

We want to learn how each voxel responds to the stimulus. For each voxel, we can write the linear model equation as:

\begin{align}
y^{v}_j =  X_j \beta^v +\epsilon_j^v
\end{align}

and:

\begin{align}
Y^{v} =  {\bf X} \beta^v +\epsilon^v
\end{align}

Since this model exist for every function, we can write it as a multiple regression function:
\begin{align}
{\bf Y} =  {\bf X} \boldsymbol\beta +\boldsymbol\epsilon
\end{align}

In the above:
- ${\bf Y}$ is n x nVoxels
- ${\bf X}$ is n x d
- ${\boldsymbol \beta}$ is d x nVoxels

Let's try to minimize the **sum of squared errors (SSE)** like before with respect to $\boldsymbol\beta$, we first find the derivative:

\begin{align}
\frac{\delta \ error}{\delta \boldsymbol \beta} &=& \frac{\delta ||{\bf Y} - {\bf X} \boldsymbol \beta||_2^2}{\delta  \boldsymbol \beta}\\
 &=& -2{\bf X}^\top ({\bf Y} - {\bf X} \boldsymbol \beta)\\
\end{align}

The minimum is achieved when the derivative is zero:

\begin{align}
-2{\bf X}^\top ( {\bf Y} - {\bf X} \boldsymbol\beta) = 0\\
{\bf X}^\top {\bf Y} = {\bf X}^\top{\bf X}  \boldsymbol\beta\\
\boldsymbol \beta = ({\bf X}^\top{\bf X})^{-1}{\bf X}^\top {\bf Y}\\
\end{align}

This solution is similar to the single dimensional output solution. The first term $({\bf X}^\top{\bf X})^{-1}{\bf X}^\top$ is independent of the data. If we are estimating the parameters for one voxel, or for a large number, this term will be the same. 

Notice also that each voxel's parameters are estimated independently from each other: each column of $\boldsymbol \beta$ corresponds to the parameters of one voxel $v$, and it is obtained by multipling the matrix $({\bf X}^\top{\bf X})^{-1}{\bf X}^\top$ with the ${\bf Y}$ column that corresponds to voxel $v$.

#### Now let's continue applying linear models to real fMRI data. As introduced in last lecture, this time we'll use a linear regression estimator from scikit-learn to get another glimpse at this software package. We'll fit the faces response vector to our FFA voxel and see how we do.



In [ ]:
from sklearn.linear_model import LinearRegression

conv_X_faces = conv_X[:, 1]
# get the faces voxels
Y_faces = Y[:, 3464]

reg_model_faces = LinearRegression()
reg_model_faces.fit(conv_X_faces, Y_faces)

Uh-Oh, why didn't that work? Remember that we need to reshape vectors to 1-D matrices. This is annoying now, but will become useful once we want to find regression models to multiple voxels at the same time!

In [ ]:
reg_model_faces = LinearRegression(fit_intercept=True)
reg_model_faces.fit(conv_X_faces.reshape(-1,1), Y_faces.reshape(-1,1))

Cool, that did work now!

OK. Where is the info hidden? In all scikit-learn estimators, the properties computed during `fit` are stored in names with *trailing underscore*. In this case it is `.intercept_` and `.coef_`.

In [ ]:
reg_model_faces.intercept_, reg_model_faces.coef_

As we can see, these values correspond to `slope` and `intercept` computed above. 

Let's see how good this model is by calculating the sum of squared errors (SSE):

In [ ]:
data_faces_pred_faces = reg_model_faces.predict(conv_X_faces.reshape(-1,1))[:,0]

faces_sse = np.sum((Y_faces - data_faces_pred_faces) **2)
faces_sse

Let's plot the data and the regression line to see how well it fits.

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.plot(conv_X_faces, data_faces_pred_faces, 'k', label='Regression Line')
plt.plot(conv_X_faces, Y_faces, '.r', label='Face Data')
plt.xlabel('Faces Response')
plt.ylabel('BOLD Signal')
plt.title('FFA Voxel Regression')
_ = plt.legend()
plt.show()


#### Breakout Session

1\. Fit a linear regression model to the PPA voxel BOLD data using the `places` response vector. Plot the data as a scatter plot, and the regression line.

In [ ]:
conv_X_places = 
# get the faces voxels
Y_places = Y[:, 10433]

# Multiple Linear Regression

The real power of regression comes when we have more than one **independent variable** we want to associate with the **dependent variable**.

The case of multiple linear regression is a natural extension of the simple linear regression case. In multiple regression, we have multiple independent variables ($x^i$), and still a single dependent variable (y). Now the equation describing each time point of data looks like this: 

\begin{align}
y_j =  w_0 + w_1 x^1_j + w_2 x^2_j + \dots + w_d x^d_j +\varepsilon_j
\end{align}

We can rewrite this second equation for multiple regression using a more succinct notation:

\begin{align}
Y =  {\bf X}W + w_0 +\varepsilon,
\end{align}

where
- $Y$ is the n x 1 array of BOLD data
- ${\bf X}$ is the n x d matrix were each of the columns is a response vector (${\bf X}_{ji} = x^i_j$)
- ${W}$ is d x 1, contains all the $w_i$
- $\varepsilon$ is the n x 1 vector of errors, or residuals

We can write the solution to this multiple regression model as the following equation, which is what we will use to fit our model. This is just for your own information - you don't need to know this equation for any exams:

\begin{align}
\hat W = ({\bf X}^\top{\bf X})^{-1}{\bf X}^\top Y\\
\end{align}

Even though the above formula is not necessary for exams, take a good look at it, and try to peel apart what it means. It is likely that you will encounter it more than once throughout your career. Disregard the $({\bf X}^\top{\bf X})^{-1}$ for a bit and take a look at the rest (${\bf X}^\top Y$). Does it ring a bell? If not, refer back to the non-exam-relevant interlude from above.

As it turns out, the equation that minimizes the Sum of Squared Errors (SSE) is exactly the same for the multiple regression case as it is for the simple linear regression case, so we can use the same `LinearRegression` function for both! Let's explore this...

## Extending the Face Model: Modeling Faces and Bodies

Since we know that the subject saw more than just images of faces, it makes sense that we would want to model more than just the response to faces in order to get a model that better explains the BOLD data (y). Now let's see what happens if we include the bodies response vector in the model as well...

### Intuition using time series plots

First let's plot the FFA BOLD data along with the response vectors for the faces and bodies to remind ourselves what the signal looks like when bodies were shown.

In [ ]:
conv_X_bodies = conv_X[:, 0]
_ = plt.figure(figsize=(10, 8))
_ = plt.plot(Y_faces, label = 'Bold Data')
_ = plt.plot(conv_X_faces, label = 'Faces')
_ = plt.plot(conv_X_bodies, label = 'Bodies')
_ = plt.legend()
plt.show()

Now we need to create the **design matrix**, for just bodies and faces. We'll use `np.stack` to do that. 

In [ ]:
X_faces_bodies = np.stack((conv_X_faces, conv_X_bodies), axis=1)
print(X_faces_bodies.shape)

plt.figure()
plt.imshow(X_faces_bodies, aspect='auto')
plt.show()

And reshape the FFA voxel BOLD data into a 2-D matrix with one column.

In [ ]:
Y_faces_2D = Y_faces.reshape(len(Y_faces),1)

### Fit the model

Now let's fit a multiple regression model using both the faces and the bodies response vectors as independent variables. We just learned that the same equation estimates the coefficients for simple and multiple linear regression, so we can use the same `LinearRegression` object to fit this multiple regression model. Instead of reshaping a single response vector into a 2-D matrix, we'll use the design matrix we just created. We'll use the reshaped FFA voxel BOLD data and fit the model.

In [ ]:
reg_model_face_body = LinearRegression()
reg_model_face_body.fit(X_faces_bodies, Y_faces_2D)

Let's have a look at the coefficient for faces from this model and from the first regression model of faces that we built. There are 2 slope coefficients now, and since the faces response vector was in the first column, the first coefficient is for faces.

In [ ]:
print('Faces Coefficient with Bodies: %.03f' % (reg_model_face_body.coef_[0][0]))
print('Faces Coefficient Only: %.03f' % (reg_model_faces.coef_[0][0]))

The two coefficients are similar, but not the same! We'll learn more about why this is later on. 

For now let's predict the original data using this faces and bodies model, and then calculate the sum of squared error (SSE) of this new model. Then we can compare this SSE value with the SSE we calculated above for the faces only model. 

In [ ]:
data_faces_pred_facebody = reg_model_face_body.predict(X_faces_bodies)[:, 0]
faces_bodies_sse = np.sum((Y_faces - data_faces_pred_facebody) ** 2)
faces_bodies_sse, faces_sse

Since the SSE for the faces bodies model is lower than that for the faces only model we can conclude that including the bodies response vector into the model improves the fit!

## The Full Model with All the Categories

We saw that the model that incorporated bodies and faces did a better job than the model which just incorporated faces. Let's see what happens when we include the response vectors that describe all the stimuli that the subject saw.

We'll do this for 5 voxels that are selective to the different stimulus types.

In [ ]:
voxel_indices = np.array([16521, 3464, 8701, 8318, 8806, 10433])
five_voxels = Y[:, voxel_indices]
five_voxels.shape

Let's plot the time series of these 5 voxels to get an idea of what they look like.

In [ ]:
plt.figure(figsize=(10, 8))
for i in range(len(voxel_indices)):
    timeseries = five_voxels[:, i]
    plt.plot(timeseries + 5 * i)    
plt.show()

Now we'll fit a multiple regression model in the same way.

In [ ]:
reg_model_full = LinearRegression()
reg_model_full.fit(conv_X, five_voxels)

And let's predict and calculate the SSE again for the faces voxel, so we can compare it with the faces bodies model.

In [ ]:
data_faces_pred_full = reg_model_full.predict(conv_X)[:,1]
full_sse = np.sum((Y_faces-data_faces_pred_full)**2)
print('Just using Faces and bodies, SSE is: %.02f, adding all categories SSE is: %.02f' % (faces_bodies_sse, full_sse))

It seems the SSE is slightly lower when adding all of the response vectors, but not by much! What matters in statistics is whether this is **significantly** lower. We'll learn about significance testing in the next lecture. 

Because adding the remaining regressors does not seem to reduce the error very much anymore, it seems unlikely that our voxel responds significantly to the added categories. However, using the full set of response vectors in regression is going to be useful when working with the full brain - there may be voxels that are very well modeled by these additional response vectors.

Finally, let's plot the real BOLD data time series along with the predicted time series from using multiple regression to visualize how well the model fits.

In [ ]:
predictions = reg_model_full.predict(conv_X)

weights = reg_model_full.coef_
intercepts = reg_model_full.intercept_

plt.figure(figsize=(10, 8))
for i in range(len(voxel_indices)):
    timeseries = five_voxels[:, i]
    prediction = predictions[:, i]
    plt.plot(timeseries + 5 * i, lw=2)
    plt.plot(prediction + 5 * i, lw=2)
    
    # Now let's display the individual regressors
    for j in range(5):
        plt.plot(conv_X[:,j] * weights[i, j] + intercepts[i] + 5 * i, lw=.5)
plt.show()

## Why Do We Need Multiple Regression?

We just saw that simple and multiple linear regression don't always estimate the same value for coefficients of the same $x$ variable (the faces response). So why would we want to use multiple regression instead of estimating the coeffcicents of a number of simple linear regression models?

The answer is that when there is a correlation between the $x$ variables, multiple regression can account for that correlation, whereas doing several simple regressions cannot. In other words, multiple regression controls for correlation in the independent variables. This leads to a more accurate estimation of all the slopes.
